In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import cv2

import glob
import os
import boto3
from sqlalchemy import create_engine, MetaData, Table, select, and_, func
from sqlalchemy.orm import sessionmaker, relationship, join
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import Table, Column, Integer, ForeignKey
from sqlalchemy.orm import relationship
from aquabyte.optics import convert_to_world_point, depth_from_disp, pixel2world, euclidean_distance
from aquabyte.database_access_utils import extract_from_database

import pickle
from PIL import Image, ImageDraw

pd.set_option('max_columns', 500)
pd.set_option('max_colwidth', 50)

<h1> Establish connection to database and perform query for base dataset </h1>

In [ ]:
# AWS credentials
aws_credentials = json.load(open(os.environ["AWS_CREDENTIALS"]))
s3_client = boto3.client('s3', aws_access_key_id=aws_credentials["aws_access_key_id"],
                         aws_secret_access_key=aws_credentials["aws_secret_access_key"],
                         region_name="eu-west-1")


# prod SQL credentaials
sql_credentials = json.load(open(os.environ["PROD_SQL_CREDENTIALS"]))

sql_query = '''
select * from keypoint_annotations
where captured_at >= '2019-05-15'
and site_id = 23
and pen_id = 4;
'''

original_df = extract_from_database(sql_query, sql_credentials)
original_df = original_df.loc[:, ~original_df.columns.duplicated()]
        

<h1> Iterate over query results and generate 3D coordinates + biomass estimates for each stereo fish detection </h1>

In [ ]:
def coord2biomass_linear(world_keypoints, model):
    """from coordinates to biomass"""

    mean = model['mean']
    std= model['std']
    PCA_components = model['PCA_components']
    reg_coef = model['reg_coef']
    reg_intercept = model['reg_intercept']
    body_parts = model['body_parts']

    # calculate pairwise distances for production coord
    # based on the exact ordering reflected in the body_parts
    # variable above

    pairwise_distances = []
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            dist = euclidean_distance(world_keypoints[body_parts[i]], world_keypoints[body_parts[j]])
            pairwise_distances.append(dist)
    
    interaction_values = []
    for i in range(len(pairwise_distances)):
        for j in range(i, len(pairwise_distances)):
            dist1 = pairwise_distances[i]
            dist2 = pairwise_distances[j]
            interaction_values.append(dist1 * dist2)

    X = np.array(pairwise_distances + interaction_values)

    X_normalized = (X - model['mean']) / model['std']
    X_transformed = np.dot(X_normalized, model['PCA_components'].T)
    prediction = np.dot(X_transformed, reg_coef) + reg_intercept
    return prediction


def coord2biomass_blender(world_keypoints, blender):
    """from coordinates to biomass"""

    reverse_mapping = blender["reverse_mapping"]
    distances = np.array(blender["distances"])
    volumes = blender["volume"]
    regression_coeff = blender["coeff"]

    # calculate pairwise distances for production coord
    # the reverse mapping insure that we listing the kp
    # in the same order
    measurements = []
    number_of_parts = len(world_keypoints)
    for k in range(number_of_parts):
        v = world_keypoints[reverse_mapping[str(k)]]
        for k0 in range(k+1, number_of_parts):
            v0 = world_keypoints[reverse_mapping[str(k0)]]
            dist = euclidean_distance(v, v0)*1000 # mm to m
            measurements.append(dist)
    measurements = np.array(measurements)

    # absolute diff
    diff = np.nanmean(np.abs(distances - measurements), axis=1)
    closest = np.argmin(diff)
    prediction = volumes[closest]

    # here is some machine learning
    prediction = prediction*regression_coeff[0] + regression_coeff[1]
    return prediction



In [ ]:
# load model parameters for Blender and linear models
model = pickle.load(open('/root/data/alok/biomass_estimation/models/model.pkl', 'rb'))
blender = json.load(open('/root/data/alok/biomass_estimation/models/volumes.json'))


# establish new columns
mask = (original_df.is_skipped == False) & (~original_df.keypoints.isnull())
for col in ['left_keypoints', 'right_keypoints', 'world_keypoint_coordinates']:
    original_df[col] = np.nan
    original_df[col] = original_df[col].astype(object)
for col in ['predicted_biomass_linear', 'predicted_biomass_blender', 'max_y_coordinate_deviation']:
    original_df[col] = np.nan


# modify the dataframe row-by-row
for idx, row in original_df[mask].iterrows():
    keypoints = row.keypoints
    left_keypoints = keypoints['leftCrop']
    right_keypoints = keypoints['rightCrop']
            
    # compute world coordinates
    camera_metadata = row.camera_metadata
    camera_metadata['pixelCountHeight'] = 3000
    camera_metadata['pixelCountWidth'] = 4096
    world_keypoint_coordinates = pixel2world(left_keypoints, right_keypoints, camera_metadata)
    
    # update dataframe with world keypoint coordinates
    original_df.at[idx, 'left_keypoints'] = left_keypoints
    original_df.at[idx, 'right_keypoints'] = right_keypoints
    original_df.at[idx, 'world_keypoint_coordinates'] = world_keypoint_coordinates
    
    body_parts = sorted(list(world_keypoint_coordinates.keys()))
    for i in range(len(body_parts)-1):
        for j in range(i+1, len(body_parts)):
            bp1, bp2 = body_parts[i], body_parts[j]
            col = '{}<->{}'.format(body_parts[i], body_parts[j])
            if not col in original_df.columns:
                original_df[col] = np.nan
            original_df.at[idx, col] = \
                euclidean_distance(world_keypoint_coordinates[bp1], world_keypoint_coordinates[bp2])
    
    # update dataframe with biomass predictions from both models
    predicted_biomass_linear = coord2biomass_linear(world_keypoint_coordinates, model)
    predicted_biomass_blender = coord2biomass_blender(world_keypoint_coordinates, blender)
    original_df.at[idx, 'predicted_biomass_linear'] = predicted_biomass_linear
    original_df.at[idx, 'predicted_biomass_blender'] = predicted_biomass_blender
    
    # update dataframe with keypoint deviation
    threshold = 10
    left_keypoint_y_coords = {bp['keypointType']: bp['yFrame'] for bp in left_keypoints}
    right_keypoint_y_coords = {bp['keypointType']: bp['yFrame'] for bp in right_keypoints}
    max_y_coordinate_deviation = \
        max([abs(left_keypoint_y_coords[bp] - right_keypoint_y_coords[bp]) for bp in body_parts])
    
    original_df.at[idx, 'max_y_coordinate_deviation'] = max_y_coordinate_deviation



<h1> Apply filters </h1>

In [ ]:
invalid_fish_detection_ids = json.load(open('/root/data/alok/biomass_estimation/invalid_fish_detection_ids.json'))
df = original_df.copy(deep=True)

# define filters
valid_linear_prediction_mask = ~df.predicted_biomass_linear.isnull()
rectification_valid_mask = (~df.fish_detection_id.isin(invalid_fish_detection_ids))
keypoints_valid_mask = (df.max_y_coordinate_deviation < 15)
qa_mask = df.is_qa = True
mask = valid_linear_prediction_mask & rectification_valid_mask & keypoints_valid_mask & qa_mask

df = df[mask].copy(deep=True)
df.index = pd.to_datetime(df.captured_at)



<h1> Pairwise Distance Distributions </h1>

Here, we plot the distributions of the pairwise distance features. This will allow us to visibly spot and potentially diagnose bad biomass estimates, and give more insights as to why the linear PCA model may be failing.

In [ ]:
body_parts = sorted([item['keypointType'] for item in df.left_keypoints.iloc[0]])
N = len(body_parts)
fig, axes = plt.subplots(N, N, figsize=(30, 30))
i = 1
for i, bp1 in enumerate(body_parts):
    for j, bp2 in enumerate(body_parts):
        if bp1 >= bp2:
            continue
        axes[i, j].scatter(df['{}<->{}'.format(bp1, bp2)], df['TAIL_NOTCH<->UPPER_LIP'])
        axes[i, j].grid()
        axes[i, j].set_title('{}<->{}'.format(bp1, bp2))
        
        

In [ ]:
plt.hist(df.max_y_coordinate_deviation, bins=100)
plt.show()

<h1> Compute biomass estimate using Linear Model + PCA + interaction features for pairwise distances </h1>

In [ ]:
df.sort_values('predicted_biomass_linear', ascending=False)

In [ ]:
plt.hist(df.predicted_biomass_linear, bins=100)
plt.xlabel('Predicted biomass')
plt.ylabel('Count')

<h1> Examine the worst cases </h1>

In [ ]:
def plot_coordinates(image_url, side, keypoints):
    image_f = './image.jpg'
#     bucket = 'aquabyte-frames-resized-inbound'
    bucket = 'aquabyte-crops'
    key = image_url[image_url.index('aquabyte-crops') + len('aquabyte-crops') + 1:]
    s3_client.download_file(bucket, key, image_f)
    plt.figure(figsize=(30, 10))
    im = plt.imread(image_f)
    
    for keypoint in keypoints:
        keypoint_type = keypoint['keypointType']
        x, y = keypoint['xCrop'], keypoint['yCrop']
        plt.scatter([x], [y])
        plt.annotate(keypoint_type, (x, y), color='red')
        
    plt.imshow(im)        
    

In [ ]:
keypoint_annotation_id = 97971
keypoint_annotation_mask = (df.id == keypoint_annotation_id)
left_image_url = df[keypoint_annotation_mask].left_image_url.iloc[0]
left_keypoints = df[keypoint_annotation_mask].left_keypoints.iloc[0]
right_image_url = df[keypoint_annotation_mask].right_image_url.iloc[0]
right_keypoints = df[keypoint_annotation_mask].right_keypoints.iloc[0]

world_keypoint_coordinates = df[keypoint_annotation_mask].world_keypoint_coordinates.iloc[0]
im_left = plot_coordinates(left_image_url, 'left', left_keypoints)
im_right = plot_coordinates(right_image_url, 'right', right_keypoints)

In [ ]:
left_keypoint_y_coords = sorted([(item['keypointType'], item['yFrame']) for item in left_keypoints])
right_keypoint_y_coords = sorted([(item['keypointType'], item['yFrame']) for item in right_keypoints])
left_keypoint_x_coords = sorted([(item['keypointType'], item['xFrame']) for item in left_keypoints])
right_keypoint_x_coords = sorted([(item['keypointType'], item['xFrame']) for item in right_keypoints])
left_keypoint_y_coords, right_keypoint_y_coords

In [ ]:
world_keypoint_coordinates

In [ ]:
def load_params(params_file):
    params = json.load(open(params_file))
    cameraMatrix1 = np.array(params['CameraParameters1']['IntrinsicMatrix']).transpose()
    cameraMatrix2 = np.array(params['CameraParameters2']['IntrinsicMatrix']).transpose()

    distCoeffs1 = params['CameraParameters1']['RadialDistortion'][0:2] + \
                   params['CameraParameters1']['TangentialDistortion'] + \
                   [params['CameraParameters1']['RadialDistortion'][2]]
    distCoeffs1 = np.array(distCoeffs1)

    distCoeffs2 = params['CameraParameters2']['RadialDistortion'][0:2] + \
                   params['CameraParameters2']['TangentialDistortion'] + \
                   [params['CameraParameters2']['RadialDistortion'][2]]
    distCoeffs2 = np.array(distCoeffs2)

    R = np.array(params['RotationOfCamera2']).transpose()
    T = np.array(params['TranslationOfCamera2']).transpose()

    imageSize = (4096, 3000)
    
    # perform rectification
    (R1, R2, P1, P2, Q, leftROI, rightROI) = cv2.stereoRectify(cameraMatrix1, distCoeffs1, cameraMatrix2, distCoeffs2, imageSize, R, T, None, None, None, None, None, cv2.CALIB_ZERO_DISPARITY, 0)

    left_maps = cv2.initUndistortRectifyMap(cameraMatrix1, distCoeffs1, R1, P1, imageSize, cv2.CV_16SC2)
    right_maps = cv2.initUndistortRectifyMap(cameraMatrix2, distCoeffs2, R2, P2, imageSize, cv2.CV_16SC2)
    
    return left_maps, right_maps

def get_remap(crop, side, crop_metadata, stereo_params_f):
    left_maps, right_maps = load_params(stereo_params_f)
    maps = left_maps if side == 'left' else right_maps
    new_image = np.zeros([3000, 4096, 3]).astype('uint8')
    lower_left = (crop_metadata['y_coord'] + crop_metadata['height'], crop_metadata['x_coord'])
    upper_right = (crop_metadata['y_coord'], crop_metadata['x_coord'] + crop_metadata['width'])
    new_image[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :] = np.array(crop)
    remap = cv2.remap(new_image, maps[0], maps[1], cv2.INTER_LANCZOS4)
    nonzero_indices = np.where(remap > 0)
    y_min, y_max = nonzero_indices[0].min(), nonzero_indices[0].max() 
    x_min, x_max = nonzero_indices[1].min(), nonzero_indices[1].max()
    lower_left = (y_max, x_min)
    upper_right = (y_min, x_max)
    rectified_crop = remap[upper_right[0]:lower_left[0], lower_left[1]:upper_right[1], :].copy()
    print(crop_metadata)
    rectified_crop_metadata = crop_metadata.copy()
    rectified_crop_metadata['x_coord'] = x_min
    rectified_crop_metadata['y_coord'] = y_min
    rectified_crop_metadata['width'] = x_max - x_min
    rectified_crop_metadata['height'] = y_max - y_min
    
    return remap, rectified_crop_metadata



In [ ]:
stereo_params_f = './2019-04-26_blom_kjeppevikholmen_pen_1.json'
left_crop_metadata = df[fish_id_mask].left_crop_metadata.iloc[0]
left_new_image = np.zeros([3000, 4096, 3]).astype('uint8')
left_remap, rectified_left_crop_metadata = get_remap(im_left, 'left', left_crop_metadata, stereo_params_f)

right_crop_metadata = df[fish_id_mask].right_crop_metadata.iloc[0]
right_new_image = np.zeros([3000, 4096, 3]).astype('uint8')
right_remap, rectified_right_crop_metadata = get_remap(im_right, 'right', right_crop_metadata, stereo_params_f)

In [ ]:
[(kp['keypointType'], kp['yCrop'] + rectified_left_crop_metadata['y_coord']) for kp in left_keypoints]

In [ ]:
[(kp['keypointType'], kp['yCrop'] + rectified_right_crop_metadata['y_coord']) for kp in right_keypoints]

In [ ]:
Image.fromarray(right_remap).save('./right_remap.jpg')
Image.fromarray(left_remap).save('./left_remap.jpg')

In [ ]:
left_keypoints

In [ ]:
right_keypoints

In [ ]:
x

In [ ]:
df['2019-05-02']['predicted_biomass_blender'].mean()

In [ ]:
mask = (df.site_id == 23) & (df.pen_id == 4) & (df.index >= '2019-04-27')
df[mask].predicted_biomass_blender.resample('D', how=lambda x: x.mean())

In [ ]:
plt.hist(df[mask].predicted_biomass_blender)

In [ ]:
df[mask].shape

In [ ]:
mask = (df.predicted_biomass_linear > 500) & (df.predicted_biomass_linear < 6000)
plt.scatter(df.ix[mask, 'predicted_biomass_blender'], df.ix[mask, 'predicted_biomass_linear'])
plt.show()

<h1> Examine rectification issue </h1>

In [ ]:
df.shape

In [ ]:
df = pd.read_csv('./data_dump.csv')

In [ ]:
rectified_bucket = 'aquabyte-crops'
left_image_rectified_f = './left_image_rectified.jpg'
right_image_rectified_f = './right_image_rectified.jpg'

invalid_fish_detection_ids, invalid_urls = [], []
i = 0
for idx, row in df.iterrows():
    if i % 100 == 0:
        print(i)
    i += 1
    if i < 36132:
        continue
    left_image_url = row.left_image_url
    right_image_url = row.right_image_url
    left_rectified_key = left_image_url[left_image_url.index('aquabyte-crops') + len('aquabyte-crops') + 1:]
    s3_client.download_file(rectified_bucket, left_rectified_key, left_image_rectified_f)
    right_rectified_key = right_image_url[right_image_url.index('aquabyte-crops') + len('aquabyte-crops') + 1:]
    s3_client.download_file(rectified_bucket, right_rectified_key, right_image_rectified_f)
    
    # this is dumb, can probably do this in memory
    left_rectified_image = cv2.imread(left_image_rectified_f)
    right_rectified_image = cv2.imread(right_image_rectified_f)
    
    left_crop_metadata = json.loads(row.left_crop_metadata)
    right_crop_metadata = json.loads(row.right_crop_metadata)
    left_crop_width = left_crop_metadata['width']
    left_crop_height = left_crop_metadata['height']
    right_crop_width = right_crop_metadata['width']
    right_crop_height = right_crop_metadata['height']
    
    invalid = False
    if left_rectified_image.shape[0] == left_crop_height and left_rectified_image.shape[1] == left_crop_width:
        invalid = True
        invalid_urls.append(left_image_url)
        print('left image not rectified for id {}!'.format(row.id))
    if right_rectified_image.shape[0] == right_crop_height and right_rectified_image.shape[1] == right_crop_width:
        invalid = True
        invalid_urls.append(right_image_url)
        print('right image not rectified for id {}!'.format(row.id))
    
    if invalid:
        invalid_fish_detection_ids.append(int(row.id))
    
    
    
        
    
        

In [ ]:
pickle.dump(invalid_ids, open('./invalid_ids', 'wb'))

In [ ]:
i

In [ ]:
json.dump(invalid_urls + invalid_urls_old, open('./invalid_urls.json', 'w'))

In [ ]:
invalid_urls_old = json.load(open('./invalid_urls.json'))

In [ ]:
invalid_fish_detection_ids_old = json.load(open('./invalid_fish_detection_ids.json'))

In [ ]:
json.dump(invalid_fish_detection_ids + invalid_fish_detection_ids_old, open('./invalid_fish_detection_ids.json', 'w'))

<h1> Prod data backfill </h1>

In [ ]:
sql_query = '''
select 

k.id, k.fish_detection_id, k.annotated_by_email, k.is_qa, 
k.is_skipped, k.is_blurry, k.is_dark, k.is_occluded,
k.is_bad_orientation, k.is_partial, k.direction, k.keypoints, 
k.work_duration_left_ms, k.work_duration_right_ms, f.created_at, 
f.updated_at, f.captured_at, f.site_id, f.pen_id, f.left_image_url, f.right_image_url, 
f.left_crop_metadata, f.right_crop_metadata, f.camera_metadata

from keypoint_annotations k
left join fish_detections f
on k.fish_detection_id = f.id
'''

df = extract_from_database(sql_query, sql_credentials)


In [ ]:
for idx, row in df.iterrows():
    if row.keypoints:
        keypoints = row.keypoints
        left_crop_metadata = row.left_crop_metadata
        right_crop_metadata = row.right_crop_metadata
        
        for 
        
        break

In [ ]:
left_keypoints